In [2]:
library(Biostrings)

Warning message:
"package 'Biostrings' was built under R version 4.3.3"
Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: 'S4Vectors'


The following object is masked from 'package:utils':

    findMatches


The following objects are masked from 'package:base':

    expand.grid, I, unname


Loading required package: IRanges


Attaching p

In [4]:
DNA <- readDNAStringSet("seq_motif.fasta")

In [5]:
DNA

DNAStringSet object of length 4:
    width seq                                               names               
[1]     8 AACGTGCT                                          sequence 1
[2]     8 CATACGTA                                          sequence 2
[3]     8 ACCTTAGC                                          sequence 3
[4]     8 TCTCCGTA                                          sequence 4

# The Brute-Force Motif Search

## task 1

In [7]:
Score <- function(s, DNA, l) {
  # Initialize the frequency profile matrix
  profile <- matrix(0, nrow = 4, ncol = l, dimnames = list(c("A", "C", "G", "T")))
  
  # Fill in the profile matrix by counting nucleotides
  for (i in seq_along(s)) {
    start <- s[i]
    motif <- subseq(DNA[[i]], start, start + l - 1)
    for (j in 1:l) {
      nucleotide <- as.character(motif[j])
      profile[nucleotide, j] <- profile[nucleotide, j] + 1
    }
  }
  
  # Calculate the score by summing the highest counts in each column (consensus string score)
  score <- sum(apply(profile, 2, max))
  return(score)
}

s <- c(1, 1, 1, 1)
l <- 8
Score(s, DNA, l)

[1] 16

## task 2

In [9]:
NextLeaf <- function(a, L, k) {
  for (i in L:1) {
    if (a[i] < k) {
      a[i] <- a[i] + 1
      return(a)
    }
    a[i] <- 1
  }
  return(a)
}

## task 3

In [13]:
BFMotifSearch <- function(DNA, t, n, l) {
  # Step 1: Initialize starting positions array s and bestScore
  s <- rep(1, t)
  bestScore <- Score(s, DNA, l)
  bestMotif <- s
  
  # Maximum starting index k for each sequence
  k <- n - l + 1
  
  # Step 2: Search through all starting positions using NextLeaf
  repeat {
    # Generate the next leaf in the enumeration tree
    s <- NextLeaf(s, t, k)
    
    # Calculate score for the current starting positions
    currentScore <- Score(s, DNA, l)
    
    # Update bestScore and bestMotif if a higher score is found
    if (currentScore > bestScore) {
      bestScore <- currentScore
      bestMotif <- s
    }
    
    # Check if we've looped back to the initial position
    if (all(s == rep(1, t))) {
      break
    }
  }
  
  # Step 3: Return the best starting positions (bestMotif) for the highest score
  return(bestMotif)
}


t <- length(DNA)

# Length of each DNA sequence
n <- width(DNA[1])

# Motif length
l <- 5

# Run the brute-force motif search
bestMotif <- BFMotifSearch(DNA, t, n, l)
print(bestMotif)

[1] 2 4 1 4


# The Branch-and-Bound Motif Search

## task 4

In [14]:
NextVertex <- function(a, i, L, k) {
  if (i < L) {
    # Step 1: If i < L, increment level and reset the next position to 1
    a[i + 1] <- 1
    return(list(a = a, i = i + 1))
  } else {
    # Step 2: Start from the last position and move backward
    for (j in L:1) {
      if (a[j] < k) {
        # Increment the position and reset all following positions to 1
        a[j] <- a[j] + 1
        if (j < L) {
          a[(j + 1):L] <- 1
        }
        return(list(a = a, i = j))
      }
    }
  }
  
  # Step 3: If no increment is possible, return to initial configuration with i = 0
  return(list(a = a, i = 0))
}

a <- c(1, 1, 1)  # Initial starting indexes
i <- 1           # Initial level
L <- 3           # Number of DNA sequences
k <- 4           # Maximum valid starting position index

# Run NextVertex to get the next vertex in the tree
result <- NextVertex(a, i, L, k)
print(result$a)  # Updated starting indexes
print(result$i)  # Updated level

[1] 1 1 1
[1] 2


# task 5

In [15]:
ByPass <- function(a, i, L, k) {
  # Start from level i and move backward to find a position that can be incremented
  for (j in i:1) {
    if (a[j] < k) {
      # Increment the position and return the updated array and level
      a[j] <- a[j] + 1
      return(list(a = a, i = j))
    }
  }
  
  # If no increment is possible, return a with level set to 0
  return(list(a = a, i = 0))
}

a <- c(1, 3, 4)  # Initial starting indexes
i <- 2           # Current level
L <- 3           # Number of DNA sequences
k <- 4           # Maximum valid starting position index

# Run ByPass to skip a subtree and move to the next available leaf
result <- ByPass(a, i, L, k)
print(result$a)  # Updated starting indexes
print(result$i)  # Updated level

[1] 1 4 4
[1] 2


# task 6

potřeba upravit funkci score

In [16]:
Score <- function(s, i, DNA, l) {
  # Initialize the frequency profile matrix
  profile <- matrix(0, nrow = 4, ncol = l, dimnames = list(c("A", "C", "G", "T")))
  
  # Fill in the profile matrix for the first i rows
  for (j in 1:i) {
    start <- s[j]
    motif <- subseq(DNA[[j]], start, start + l - 1)
    for (k in 1:l) {
      nucleotide <- as.character(motif[k])
      profile[nucleotide, k] <- profile[nucleotide, k] + 1
    }
  }
  
  # Calculate the score by summing the highest counts in each column (consensus string score)
  score <- sum(apply(profile, 2, max))
  return(score)
}

In [17]:
BBMotifSearch <- function(DNA, t, n, l) {
  # Step 1: Initialize starting positions array s and bestScore
  s <- rep(1, t)
  bestScore <- 0
  bestMotif <- s
  i <- 1
  
  # Maximum starting index k for each sequence
  k <- n - l + 1
  
  # Step 2: Branch-and-Bound search
  while (i > 0) {
    if (i < t) {
      # Calculate optimistic score for partial consensus up to row i
      optimisticScore <- Score(s, i, DNA, l) + (t - i) * l
      if (optimisticScore < bestScore) {
        # Skip subtree if optimistic score is less than current best score
        bypass_result <- ByPass(s, i, t, k)
        s <- bypass_result$a
        i <- bypass_result$i
      } else {
        # Move to the next vertex
        next_vertex_result <- NextVertex(s, i, t, k)
        s <- next_vertex_result$a
        i <- next_vertex_result$i
      }
    } else {
      # Calculate full score for current configuration
      currentScore <- Score(s, t, DNA, l)
      if (currentScore > bestScore) {
        bestScore <- currentScore
        bestMotif <- s
      }
      # Move to the next vertex
      next_vertex_result <- NextVertex(s, i, t, k)
      s <- next_vertex_result$a
      i <- next_vertex_result$i
    }
  }
  
  # Step 3: Return the best motif found
  return(bestMotif)
}

In [18]:
DNA <- DNAStringSet(c("AACGTGCT", "CATACGTA", "ACCTTAGC", "TCTCCGTA"))

# Number of sequences
t <- length(DNA)

# Length of each DNA sequence
n <- width(DNA[1])

# Motif length
l <- 5

# Run the Branch-and-Bound Motif Search
bestMotif <- BBMotifSearch(DNA, t, n, l)
print(bestMotif)

[1] 2 4 1 4
